# Part 2: Homework 1
*Course: Mathematics 2 (DataScience@FRI, University of Ljubljana)*

## Interior Point Method

This notebook tests the custom implementation of Interior Point Method and compares it to a commercial LP solver.

In [1]:
# Global imports
import numpy as np
from scipy.optimize import linprog

from optimizers.InteriorPointMethodMinimizer import InteriorPointMethodMinimizer

In [2]:
def is_solution_feasible(x: np.array, A: np.array, b: np.array):
    """
    Checks if solutions is feasible.

    :param x: Solution vector x
    :param A: Coefficient matrix A
    :param b: Bounds vector b
    :return: True if solution is feasible, else False.
    """
    # x >= 0
    if not all(i >= 0 for i in list(x_custom_implementation)):
        return False

    nutritional_vals = A @ x
    nutritional_vals_half = nutritional_vals[:int(len(nutritional_vals) / 2)]

    # Check if solution satisfies constraints
    for i in range(len(nutritional_vals_half)):
        upper_bound = b[i]
        lower_bound = b[i + len(nutritional_vals_half)]
        if nutritional_vals_half[i] > upper_bound or nutritional_vals_half[i] < lower_bound:
            return False

    return True

In [3]:
def beauty_print_result(A: np.array, x: np.array, c: np.array, commercial_calculation: bool):
    """
    Beautifully prints our awesome results.

    :param x: Solution vector x
    :param A: Coefficient matrix A
    :param c: Cost vector c
    :param commercial_calculation: Boolean if x was calculated by commercial LP solver.

    """
    if commercial_calculation:
        print("Commercial application solution:")
    else:
        print("Custom solution:")

    food_names = ['Potatoes', 'Bread', 'Milk', 'Eggs', 'Yoghurt', 'Vegetable oil', 'Beef', 'Strawberries']
    print("\tQuantity over a selection of foods:")
    for food_name, quantity in zip(food_names, x):
        print(f"\t\t{food_name}: {round(float(quantity), 2)}")

    print("\tNutritional values:")
    nutritional_vals = A @ x
    nutritional_vals_half = nutritional_vals[:int(len(nutritional_vals) / 2)]
    nutritional_vals_names = ['CH', 'PR', 'FT', 'EN']
    for nutritional_val_name, nutritional_val_quantity in zip(nutritional_vals_names, nutritional_vals_half):
        print(f"\t\t{nutritional_val_name}: {round(float(nutritional_val_quantity), 2)}")

    print(f"\tCost: {round(float(c @ x), 2)}")

### Simple example from lectures

In this section we try to solve a simple example from lectures.

In [4]:
A = np.array([
    [2, 2],
    [3, 1]
]).T

b = np.array([
    [140],
    [150]
])

c = np.array([
    [100],
    [80]
])

In [5]:
ipm_minimizer = InteriorPointMethodMinimizer(A=A,
                                             b=b,
                                             c=c)
x_custom_implementation = ipm_minimizer.optimize()
print(f"Custom solution: {x_custom_implementation.flatten()}")

Custom solution: [77.5 -5. ]


In [6]:
result = linprog(c=c, A_ub=A, b_ub=b)
x_commercial = result.x
print(f"Commercial solution: {x_commercial}")

Commercial solution: [0. 0.]


### A man does not live by bread alone
We are given a matrix of nutrients and corresponding nutritional values and cost. Given certain constraints we have to minimize the cost of our diet.

We test our own implementation of the Interior point method and the commercial LP solver.

In [7]:
# Columns - food
# Rows - variables that we are optimizing and that are used for contstraints
nutritional_vals_mtx = np.array([[10, 22, 15, 45, 40, 20, 87, 21],
                                 [18, 48, 5, 1, 5, 0, 0, 8],
                                 [2, 11, 3, 13, 3, 0, 15, 1],
                                 [0, 5, 3, 10, 3, 100, 30, 1],
                                 [77, 270, 60, 140, 61, 880, 330, 32]])

# Since we have an upper and lower bound, we have to do some transformations.
# The idea is to separate the lower and upper bounds into two inequalities and negating the lower bound
# inequality, thus providing additional bounds that are a linear combination of negative nutritional values.

# Upper, lower bounds vector
b = np.array([370, 170, 90, 2400, -250, -50, -50, -2200])

# Upper, lower bounds coeeficient matrix A
A = np.concatenate(
    (nutritional_vals_mtx[1:], -nutritional_vals_mtx[1:]),
    axis=0
)

# Cost vector
c = nutritional_vals_mtx[0]


In [8]:
for forced_delta in np.arange(0.8, 0.95, 0.01):
    for norm_integer in np.arange(100, 1000, 100):
        ipm_minimizer = InteriorPointMethodMinimizer(A=A,
                                                     b=b,
                                                     c=c,
                                                     norm_integer=norm_integer,
                                                     forced_delta=forced_delta)
        x_custom_implementation = ipm_minimizer.optimize()
        if is_solution_feasible(x=x_custom_implementation, A=A, b=b):
            print(f"Forced delta: {forced_delta}, normalization integer: {norm_integer}")
            beauty_print_result(A=A, x=x_custom_implementation, c=c, commercial_calculation=False)

Forced delta: 0.9200000000000002, normalization integer: 100
Custom solution:
	Quantity over a selection of foods:
		Potatoes: 0.0
		Bread: 6.47
		Milk: 0.0
		Eggs: 3.95
		Yoghurt: 0.0
		Vegetable oil: 0.0
		Beef: 0.0
		Strawberries: 0.0
	Nutritional values:
		CH: 314.36
		PR: 122.45
		FT: 71.81
		EN: 2298.68
	Cost: 319.89
Forced delta: 0.9400000000000002, normalization integer: 900
Custom solution:
	Quantity over a selection of foods:
		Potatoes: 0.0
		Bread: 6.81
		Milk: 0.0
		Eggs: 0.0
		Yoghurt: 0.0
		Vegetable oil: 0.52
		Beef: 0.0
		Strawberries: 0.0
	Nutritional values:
		CH: 326.78
		PR: 74.89
		FT: 86.32
		EN: 2298.15
	Cost: 160.23


In [13]:
result = linprog(c=c, A_ub=A, b_ub=b)
x_commercial = result.x
beauty_print_result(A=A, x=x_commercial, c=c, commercial_calculation=True)

Commercial application solution:
	Quantity over a selection of foods:
		Potatoes: 0.0
		Bread: 6.23
		Milk: 0.0
		Eggs: 0.0
		Yoghurt: 0.0
		Vegetable oil: 0.59
		Beef: 0.0
		Strawberries: 0.0
	Nutritional values:
		CH: 299.04
		PR: 68.53
		FT: 90.0
		EN: 2200.0
	Cost: 148.83
